In [79]:
# metrics :
import sklearn.metrics as metrics
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))


In [80]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [81]:
#!ls 
#!pip install sklearn
#!pip install -U scikit-learn
#!which python

In [82]:
filepath= './'
train = pd.read_csv(filepath+'train.csv')
test = pd.read_csv(filepath+'test.csv')
print(train.columns)
print(test.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [83]:
def print_nan(train):
    for col in train.columns:
        print(col, str(round(100* train[col].isnull().sum() / len(train), 2)) + '%')

In [84]:
print_nan(train)

PassengerId 0.0%
Survived 0.0%
Pclass 0.0%
Name 0.0%
Sex 0.0%
Age 19.87%
SibSp 0.0%
Parch 0.0%
Ticket 0.0%
Fare 0.0%
Cabin 77.1%
Embarked 0.22%


In [85]:
def preprocess_data(train):
    emputed_train = train.drop(labels=['Cabin','Name','Ticket','Fare','PassengerId'], axis=1)

    emputed_train['Age'] = emputed_train['Age'].interpolate()   # i dont remember what this does

    copy = emputed_train.copy()

    emputed_train = emputed_train.dropna()
    set(emputed_train['Embarked'])

    def age_classifier(age):
        if (age < 10 ): # bebe
            return 0
        if (age < 18): # piti
            return 1
        if (age < 35): # jeune
            return 2
        if (age < 50): # grand
            return 3
        if (age < 69): # ;)
            return 4
        if (age < 80): # cheyeb
            return 5
        return 6 # kobba

    age_classes = [ age_classifier(age) for age in emputed_train['Age'] ]

    ist_ein_Mann = []
    for sex in emputed_train['Sex']:
        if (sex == 'male'):
            ist_ein_Mann.append(1)
        else :
            ist_ein_Mann.append(0)

    embarcation_classes = []
    for embarcation in emputed_train['Embarked']:
        if(embarcation == 'C'):
            embarcation_classes.append(0)
        else :
            if(embarcation == 'D'):
                embarcation_classes.append(1)
            else:
                embarcation_classes.append(2)

    emputed_train['Age'] = age_classes
    emputed_train['Sex'] = ist_ein_Mann
    emputed_train['Embarked'] = embarcation_classes
    return emputed_train

In [86]:
new_train = preprocess_data(train)

new_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,1,2,1,0,2
1,1,1,0,3,1,0,0
2,1,3,0,2,0,0,2
3,1,1,0,3,1,0,2
4,0,3,1,3,0,0,2


In [87]:
y_train = new_train['Survived'].to_numpy()
X_train =new_train.drop(labels=['Survived'], axis=1).to_numpy()

In [88]:
X_test = preprocess_data(test)

X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,1,2,0,0,2
1,3,0,3,1,0,2
2,2,1,4,0,0,2
3,3,1,2,0,0,2
4,3,0,2,1,1,2


In [89]:
# Reutilisation w fazet
def submit(model_name, predictions):
    predictions = np.asarray([int(p) for p in predictions])
    predictions = predictions.astype(np.int16)
    submission = pd.read_csv('./gender_submission.csv')
    submission['Survived'] = predictions
    submission.to_csv("titanic_"+str(model_name)+"_neshy.csv",index=False)    
    


In [90]:
# models :
# DecisionTreeRegressor:
from sklearn.tree import DecisionTreeRegressor
melbourne_model = DecisionTreeRegressor(random_state=69)

melbourne_model.fit(X_train, y_train)
predictions = melbourne_model.predict(X_test)
submit("DTR", predictions)

In [95]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

forest_model = RandomForestClassifier(random_state=69)
forest_model.fit(X_train, y_train)
predictions = forest_model.predict(X_test)

submit("RFC", predictions)

In [97]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier()
xgb_model.fit(X_train.values , y_train)

predictions = xdg_model.predict(X_test.values )
submit("XGB",predictions)

AttributeError: 'numpy.ndarray' object has no attribute 'as_matrix'